<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
%pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-core asknews langgraph
%pip install --quiet -U "langchain-community>=0.2.16" langchain-exa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8

In [2]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools import JinaSearch
from dotenv import load_dotenv
from datetime import datetime, timedelta
from exa_py import Exa
from langchain_core.tools import tool

# Load environment variables
load_dotenv()

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"
SERPER_API_KEY = "d8e815ef6caa94dbef7b977a0ea7d505b43a5a06"
EXA_API_KEY = "953b5801-11be-4b37-a313-f8df8f37027c"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET
os.environ["SERPER_API_KEY"] = SERPER_API_KEY
os.environ["EXA_API_KEY"] = EXA_API_KEY

In [3]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama-3.1-8b-instant", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Pinecone and vector store
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

# Initialize search tools
asknews_tool = AskNewsSearch(max_results=5)
google_serper = GoogleSerperAPIWrapper()
jina_search = JinaSearch()

# Initialize Exa search tools
exa = Exa(api_key=os.environ["EXA_API_KEY"])

@tool
def search_and_contents(
    query: str,
    include_domains: list[str] = None,
    exclude_domains: list[str] = None,
    start_published_date: str = None,
    end_published_date: str = None,
    include_text: list[str] = None,
    exclude_text: list[str] = None,
):
    """
    Search for webpages based on the query and retrieve their contents.

    Parameters:
    - query (str): The search query.
    - include_domains (list[str], optional): Restrict the search to these domains.
    - exclude_domains (list[str], optional): Exclude these domains from the search.
    - start_published_date (str, optional): Restrict to documents published after this date (YYYY-MM-DD).
    - end_published_date (str, optional): Restrict to documents published before this date (YYYY-MM-DD).
    - include_text (list[str], optional): Only include results containing these phrases.
    - exclude_text (list[str], optional): Exclude results containing these phrases.
    """
    return exa.search_and_contents(
        query,
        use_autoprompt=True,
        num_results=5,
        include_domains=include_domains,
        exclude_domains=exclude_domains,
        start_published_date=start_published_date,
        end_published_date=end_published_date,
        include_text=include_text,
        exclude_text=exclude_text,
        text=True,
        highlights=True,
    )

@tool
def find_similar_and_contents(
    url: str,
    exclude_source_domain: bool = False,
    start_published_date: str = None,
    end_published_date: str = None,
):
    """
    Search for webpages similar to a given URL and retrieve their contents.
    The url passed in should be a URL returned from `search_and_contents`.

    Parameters:
    - url (str): The URL to find similar pages for.
    - exclude_source_domain (bool, optional): If True, exclude pages from the same domain as the source URL.
    - start_published_date (str, optional): Restrict to documents published after this date (YYYY-MM-DD).
    - end_published_date (str, optional): Restrict to documents published before this date (YYYY-MM-DD).
    """
    return exa.find_similar_and_contents(
        url,
        num_results=5,
        exclude_source_domain=exclude_source_domain,
        start_published_date=start_published_date,
        end_published_date=end_published_date,
        text=True,
        highlights={"num_sentences": 1, "highlights_per_url": 1},
    )

tools = [search_and_contents, find_similar_and_contents]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

class SearchResult(BaseModel):
    source: str
    title: str
    snippet: str
    url: str
    date: Optional[str]
    media: Optional[List[Dict[str, str]]]

def vector_search(query: str) -> List[SearchResult]:
    results = vector_store.similarity_search(query, k=5)
    return [
        SearchResult(
            source="Vector Search",
            title=f"Result {i+1}",
            snippet=doc.page_content,
            url=doc.metadata.get("source", "No URL"),
            date=doc.metadata.get("date")
        ) for i, doc in enumerate(results)
    ]

def asknews_search(query: str) -> List[SearchResult]:
    results = asknews_tool.run({"query": query})
    return [
        SearchResult(
            source="AskNews",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results
    ]

def google_serper_search(query: str) -> List[SearchResult]:
    results = google_serper.results(query)
    return [
        SearchResult(
            source="Google Serper",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results.get("organic", [])
    ]

def jina_search(query: str) -> List[SearchResult]:
    results = jina_search.invoke({"query": query})
    return [
        SearchResult(
            source="Jina Search",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results
    ]

def exa_search(query: str) -> List[SearchResult]:
    results = search_and_contents(query)
    return [
        SearchResult(
            source="Exa Search",
            title=result.get("title", "No title"),
            snippet=result.get("snippet", "No snippet"),
            url=result.get("link", "No link"),
            date=result.get("date")
        ) for result in results
    ]

In [5]:
def execute_searches(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    searches = [
        ("Vector Search", vector_search),
        ("AskNews Search", asknews_search),
        ("Google Serper Search", google_serper_search),
        ("Jina Search", jina_search),
        ("Exa Search", exa_search)
    ]

    all_results = []
    for name, func in searches:
        try:
            results = func(query)
            all_results.extend(results)
        except Exception as e:
            state["messages"].append({"role": "tool", "content": f"{name} Error: {str(e)}"})

    # Sort results by date (if available) and relevance
    def sort_key(x):
        if isinstance(x, dict):
            return (x.get('date') is not None, x.get('date') or "", x.get('title', ""))
        elif hasattr(x, 'date') and hasattr(x, 'title'):
            return (x.date is not None, x.date or "", x.title)
        else:
            return (False, "", str(x))

    all_results.sort(key=sort_key, reverse=True)

    # Select top 10 most relevant results
    top_results = all_results[:10]

    state["messages"].append({"role": "tool", "content": "Search Results", "results": top_results})
    return state

In [6]:
import re
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    search_results = next((m["results"] for m in reversed(state["messages"]) if m["role"] == "tool" and "results" in m), [])

    prompt = ChatPromptTemplate.from_messages([(
        "system", """You are an advanced AI specializing in cybersecurity and intelligence analysis. Your task is to provide highly relevant, actionable, and up-to-date insights based on the latest cyber incidents. Follow these guidelines:

1. Analyze all search results thoroughly, prioritizing the most recent and relevant information.
2. Focus on information from reputable sources, official reports, and verified cybersecurity platforms.
3. Identify emerging patterns, trends, and potential implications of the latest cyber incidents.
4. Provide a structured response with the following sections:
   a. Key Highlights (bullet points of the most critical findings)
   b. Detailed Analysis (in-depth examination of the key points)
   c. Trends & Implications (numbered list of important trends and their potential impact)
   d. Critical Action Items (prioritized list of recommended actions)
   e. Emerging Concerns (potential future threats or evolving risks)

5. Include clear citations for ALL information using the format [Source Name](URL). Every piece of information must be linked to its source.
6. Prioritize information from the last 30 days. If using older sources, clearly state the date and explain why the information is still relevant.
7. If critical information is missing, identify gaps and suggest additional areas for investigation.
8. Maintain a comprehensive format, ensuring all key points are covered. Aim for a response between 800-1000 words.
9. Use Markdown formatting for emphasis: **bold** for section headers, and - for bullet points.
10. Include a "Sources" section at the end listing all unique sources used in your analysis.
11. If search results contain conflicting information, acknowledge the discrepancies and provide a balanced view.
12. For each action item or recommendation, explain the rationale and potential impact.

Previous conversation: {chat_history}
Human query: {input}
Search Results: {search_results}

Current date: {current_date}

Provide a structured, actionable response based on the latest findings, ensuring every piece of information is properly cited:
"""
    )])

    chain = prompt | llm

    current_date = datetime.now().strftime("%Y-%m-%d")

    response = chain.invoke({
        "input": state["messages"][-1]["content"],
        "search_results": "\n".join([f"{result.title}\n{result.snippet}\n{format_source_link(result.source, result.url)}\nDate: {result.date or 'Not specified'}\n" for result in search_results]),
        "chat_history": chat_history,
        "current_date": current_date
    })

    # Post-process the response to add highlights and ensure proper citations
    processed_response = add_highlights(response.content)
    processed_response = ensure_citations(processed_response, search_results)

    state["messages"].append({"role": "assistant", "content": processed_response})
    state["memory"] = {"chat_history": chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {processed_response}"}
    return state

def add_highlights(text: str) -> str:
    highlight_phrases = [
        "Critical vulnerability",
        "Zero-day exploit",
        "Ransomware attack",
        "Data breach",
        "Advanced Persistent Threat",
        "Supply chain attack",
        "Phishing campaign",
        "Malware outbreak",
        "Cybersecurity best practice",
        "Emerging threat"
    ]

    for phrase in highlight_phrases:
        text = re.sub(f"({phrase})", r"**\1**", text, flags=re.IGNORECASE)

    return text

def ensure_citations(text: str, search_results: List[SearchResult]) -> str:
    paragraphs = text.split('\n\n')
    cited_paragraphs = []

    for paragraph in paragraphs:
        if not re.search(r'', paragraph) and not paragraph.startswith('**'):
            paragraph += ' [Source needed]()'
        cited_paragraphs.append(paragraph)

    if not any(p.startswith('**Sources**') for p in cited_paragraphs):
        sources = set(f"- {format_source_link(result.source, result.url)}" for result in search_results)
        cited_paragraphs.append("**Sources**\n" + "\n".join(sources))

    return '\n\n'.join(cited_paragraphs)

def format_source_link(source: str, url: str) -> str:
    return f"[{source}]({url})"

# Workflow definition
workflow = StateGraph(AgentState)
workflow.add_node("execute_searches", execute_searches)
workflow.add_node("generate_response", generate_response)
workflow.add_edge("execute_searches", "generate_response")
workflow.add_edge("generate_response", END)
workflow.set_entry_point("execute_searches")
graph = workflow.compile()

def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory or {})
    return graph.invoke(state)

In [7]:
if __name__ == "__main__":
    query = "Share some details on currently active Infostealer and give me their TTPS and IOCs"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot Analysis:")
            print(message["content"])

<ipython-input-4-6f93e4ab9504>:62: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = search_and_contents(query)


AI Copilot Analysis:
**Cybersecurity Insights: Latest Developments and Emerging Trends**

**Key Highlights**
-----------------

* A recent report by Mandiant [1] reveals a significant increase in state-sponsored attacks targeting critical infrastructure, with a focus on the energy sector.
* A vulnerability in the widely used Log4j library has been exploited by threat actors to gain unauthorized access to systems [2].
* The FBI has issued a warning about the increasing use of AI-generated phishing emails, which are becoming increasingly sophisticated [3].
* A study by Cybersecurity Ventures [4] estimates that the global cost of cybercrime will reach $10.5 trillion by 2025.
* A new malware variant, dubbed "HermeticRansom," has been discovered, which uses a unique encryption method to evade detection [5].

**Detailed Analysis**
-------------------

### State-Sponsored Attacks

The report by Mandiant highlights the growing threat of state-sponsored attacks, particularly in the energy secto